In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import pprint
from time import sleep
import pandas as pd
import numpy as np
import tqdm


## Make functions for extracting certain fields for future dataframe

In [ ]:
def extract_brand_name(result):
    brands = []
    for i in result:
        try:
            brands.append(i.find('span',attrs={'class':"productSnippet__text--brand"}).text.strip())
        except:
            brands.append(np.nan)
    return brands


In [ ]:
def extract_title(result):
    titles = []
    for i in result:
        try:
            titles.append(i.find('span',attrs={'class':"productSnippet__text--name"}).text.strip())
        except:
            titles.append(np.nan)
    return titles


In [ ]:
def extract_url(result):
    urls = []
    for i in result:
        try:
            ids = []
            for a in div.find_all('a',attrs={'itemprop':"url"}):
                ids.append(a["href"]) 
            prefix = "https://www.vestiairecollective.com"
            for idd in ids:
                urls.append(prefix+str(ids))
        except:
            urls.append(np.nan)
    return urls


In [ ]:
def extract_price(result):
    prices = []
    for i in result:
        try:
            prices.append(i.find('span',attrs={'class':"productSnippet__price productSnippet__price--discount"}).text.strip()
        except:
            prices.append(np.nan)
    return prices


In [ ]:
def extract_likes(result):
    likes = []
    for i in result:
        try:
            likes.append(i.find('div',attrs={'class':"likeSnippet__count"}).text
        except:
            likes.append(np.nan)
    return likes
                         

In [ ]:
def extract_sold_out(result):
    sold = []
    for i in result:
        try: 
            if i.find('span',attrs={'class':"productSnippet__imageWrapper__soldText"}).text == 'Already sold':
                sold.append(1)
        except:
            sold.append(0)
    return sold
                        

## Web Scraping and Creating dataframes with extracted data


In [44]:
# example check how it gathers the data

brands = ['Louis%20Vuitton%2317', 'Chanel%2350',  'Herm%C3%A8s%2314']
models_lv = ['Speedy%2313-Speedy%20Bandouli%C3%A8re%232089', 'Neverfull%2315', 'Alma%20BB%232060-Alma%2314-No%C3%A9%23145']
models_cc = ['Wallet%20on%20Chain%23883', 'Timeless%2FClassique%233', 'Boy%234', '2.55%232']
models_h = ['Birkin%2025%232367-Birkin%2030%232366-Birkin%2035%238-Birkin%2040%232368', 'Kelly%2025%232370-Kelly%2028%232371-Kelly%2032%239-Kelly%2035%232372']
sold = [0, 1]

max_results_per_brand = 1010

url_template = "https://www.vestiairecollective.com/women-bags/handbags/p-{}/#brand={}_model={}_sold={}"


results = []
sold_out = []

for i, brand in enumerate(brands):
    for s in sold:
        for p in range(0, max_results_per_brand, 60):
            if i == 0:
                for model in models_lv:
                    print(p, brand, model, s)
            if i == 1:
                for model in models_cc:
                    print(p, brand, model, s)
            if i == 2:
                for model in models_h:
                    print(p, brand, model, s)

0 Louis%20Vuitton%2317 Speedy%2313-Speedy%20Bandouli%C3%A8re%232089 0
0 Louis%20Vuitton%2317 Neverfull%2315 0
0 Louis%20Vuitton%2317 Alma%20BB%232060-Alma%2314-No%C3%A9%23145 0
60 Louis%20Vuitton%2317 Speedy%2313-Speedy%20Bandouli%C3%A8re%232089 0
60 Louis%20Vuitton%2317 Neverfull%2315 0
60 Louis%20Vuitton%2317 Alma%20BB%232060-Alma%2314-No%C3%A9%23145 0
120 Louis%20Vuitton%2317 Speedy%2313-Speedy%20Bandouli%C3%A8re%232089 0
120 Louis%20Vuitton%2317 Neverfull%2315 0
120 Louis%20Vuitton%2317 Alma%20BB%232060-Alma%2314-No%C3%A9%23145 0
180 Louis%20Vuitton%2317 Speedy%2313-Speedy%20Bandouli%C3%A8re%232089 0
180 Louis%20Vuitton%2317 Neverfull%2315 0
180 Louis%20Vuitton%2317 Alma%20BB%232060-Alma%2314-No%C3%A9%23145 0
240 Louis%20Vuitton%2317 Speedy%2313-Speedy%20Bandouli%C3%A8re%232089 0
240 Louis%20Vuitton%2317 Neverfull%2315 0
240 Louis%20Vuitton%2317 Alma%20BB%232060-Alma%2314-No%C3%A9%23145 0
300 Louis%20Vuitton%2317 Speedy%2313-Speedy%20Bandouli%C3%A8re%232089 0
300 Louis%20Vuitton%23

In [ ]:
# Web scrape from Vestiare Collective webpage:
# Use for loops to get the results of each brand in a list, up to 1010 rows for each model types for certain brand ,
# dividing on sold out and online products

brands = ['Louis%20Vuitton%2317', 'Chanel%2350',  'Herm%C3%A8s%2314']
models_lv = ['Speedy%2313-Speedy%20Bandouli%C3%A8re%232089', 'Neverfull%2315', 'Alma%20BB%232060-Alma%2314-No%C3%A9%23145']
models_cc = ['Wallet%20on%20Chain%23883', 'Timeless%2FClassique%233', 'Boy%234', '2.55%232']
models_h = ['Birkin%2025%232367-Birkin%2030%232366-Birkin%2035%238-Birkin%2040%232368', 'Kelly%2025%232370-Kelly%2028%232371-Kelly%2032%239-Kelly%2035%232372']
sold = [0, 1]

max_results_per_brand = 1010

url_template = "https://www.vestiairecollective.com/women-bags/handbags/p-{}/#brand={}_model={}_sold={}"

# Additional url for extracting limited editions bags only for one brand but various models
# url_limited = "https://www.vestiairecollective.com/search/?q=limited#gender=Women%231_category=Bags%235%20%3E%20Clutch%20bags%2357-Bags%235%20%3E%20Handbags%2359-Bags%235%20%3E%20Travel%20bags%23527_categoryParent=Bags%235_brand=Louis%20Vuitton%2317_model=Alma%20Graffiti%236258-Alma%20BB%232060-Alma%2314-Capucines%23303-Metis%232062-Neverfull%2315-N%C3%A9oNo%C3%A9%20BB%234230-N%C3%A9oNo%C3%A9%232225-Onthego%234324-Pochette%20Accessoire%232049-Twist%23364-Speedy%2313-Speedy%20Bandouli%C3%A8re%232089-Speedy%20time%20trunk%232991-Keepall%2317"

results = []

for i, brand in enumerate(brands):
    for s in sold:
        for p in range(0, max_results_per_brand, 60):
# Make an if statement to pull the webpages for the first brand (which is Louis Vuitton) in our brands list            
            if i == 0:
                for model in models_lv:
                    r = requests.get(url_template.format(p, brand, model, s))
                    soup = BeautifulSoup(r.text, 'shtml.parser')
                    if soup.find('title').text == 'hCaptcha solve page':
                        break
                    for result in soup.find_all(name="div", attrs={"class":"row"}):
                        results.append(result)
                    time.sleep(5)
# Make an if statement to pull the webpages for the second brand (which is Chanel) in our brands list
            if i == 1:
                for model in models_cc:
                    r = requests.get(url_template.format(p, brand, model, s))
                    soup = BeautifulSoup(r.text, 'shtml.parser')
                    if soup.find('title').text == 'hCaptcha solve page':
                        break
                    for result in soup.find_all(name="div", attrs={"class":"row"}):
                        results.append(result)
                    time.sleep(5)
# Make third if statement to pull the webpages for the third brand (which is Hermes) in our brands list
            if i == 2:
                for model in models_h:
                    r = requests.get(url_template.format(p, brand, model, s))
                    soup = BeautifulSoup(r.text, 'shtml.parser')
                    if soup.find('title').text == 'hCaptcha solve page':
                        break
                    for result in soup.find_all(name="div", attrs={"class":"row"}):
                        results.append(result)
                    time.sleep(5)          
            

In [ ]:
# This for loop runs through each url and pulls from the unique web page the full description, model type,
# condition, material and color for each bag.

descriptions = []
conditions = []
modelss = []
colors = []
materials = []

urls = [extract_url(result) for result in results]
certain_m = ['2.55', 'Wallet', 'Timeless', 'Kelly', 'Birkin']

for url in urls:
    time.sleep(2)
    r=requests.get(url)
    soup=BeautifulSoup(r.text,'shtml.parser')
    temp = []
    if soup.find('title').text == 'hCaptcha solve page':
        print(soup.find('title').text)
        break 
    try: 
        for li in soup.find_all('li'):
            temp.append(li.text.strip().split('\n'))
            conditions.append(temp[6][0])
            modelss.append(temp[5][0])
            colors.append(temp[8][0])
            materials.append(temp[7][0])
    except:
        conditions.append(np.nan)
        modelss.append(np.nan)
        colors.append(np.nan)
        materials.append(np.nan)
# Extracting Description for cetain handbags to use if for later predictions if needed / 
# after manually checking webpages , I noticed that other models that are not in certain_m list, were not that informative  
    for m in certain_m:
        if certain_m in url:
            try:
                descriptions.append((url,soup.find('div', attrs={'class':"sellerDescription__action"}).text))
            except:
                descriptions.append(np.nan)



In [ ]:
# Forming dataframe with Dictionary comprehension list with the function's output
dictionary = [{'brand': extract_brand_name(result), 'title': extract_title(result), 'url': extract_url(result),
               'price_range': extract_price(result), 'n_likes': extract_likes(result), 
               'sold_out': extract_sold_out(result)} for result in results]


df = pd.DataFrame(dictionary)
# Create new column and assign all values to 0
df['limited'] = 0 

# First 942 rows are limited editions, assign this subset with 1 
df.iloc[0:943]["limited"] = 1

In [ ]:
# Create separate dataframe of Information for each bag 
df_bag_page = pd.DataFrame(descriptions, columns = ['url','description'])
df_bag_page['condition'] = conditions
df_bag_page['m_model'] = modelss
df_bag_page['color'] = colors
df_bag_page['material'] = materials
    

# merge two dataframes on url column to include all columns 
merged_df = pd.merge(df, df_bag_page, how='outer', on='url')

# dropping duplicates and reset index
merged_df.drop_duplicates(keep="first", inplace=True).reset_index(drop=True)

# Create a few new columns for further data munging and cleaning, position it in desired order
merged_df.insert(4, 'price', np.nan)
merged_df.insert(5, 'discount_price', np.nan)
merged_df.insert(7, 'model', np.nan)
merged_df.insert(11, 'material', np.nan)

# drop url column after merging datasets (no use for futher analysis and building the model)
merged_df.drop(['url'], axis=1, inplace=True)

# Save to csv preprocessed dataset
merged_df.to_csv("preview.csv", index = False)

## Additional hard coding column - Mean Retail Price

I want to add official retail prices of each model for each brand, grouped by model, type of the material the handbag made of and whether it's limited edition if relevant. 

I've made a research for each model I extracted with Beautiful Soup, and because this data is spread out on many sites or absent on official store website, I've found some missing values in public magazine articles, and decided to get the prices manually. So I assigned for each model its Mean Retail Price and rounded it.

In [ ]:
# Calculate Mean Retail Prices or have a certain price 

# LV models
Speedy = round((830+ 820)/2)
Speedy_B = round((1120 + 1100 + 1140 )/3)
Speedy_lim = round((1540 +1630+ 1750+ 1540+ 2740 +1540)/6)
Speedy_py = 5800
Speedy_ost = 10200
Neverfull = round((1120+ 1070+ 1020)/3)
Neverfull_lim = 1510
Neverfull_ali = 17300
Alma_cl= round((1160 +1400)/2) 
Alma_l= round((1570+ 1770)/2) 
Alma_lim = round((1850+1770)/2) 
Alma_bb = 1030
Alma_graf = 1640
Alma_bb_l = round((1190 +1370) /2)
Alma_bb_lim = round((1470 +1570)/2)
Alma_bb_ali = 13200
Pochette = round((385+ 495)/2) 
Pochette_lim = round((570 + 1100)/2)
Keepall = 1850
Capucines = 3950
Capucines_lim = 6900
Capucines_ali = 21900
Twist = 2600
Twist_py = 5800
NéoNoé_bb = round((1590 +1690)/2)
NéoNoé_lim = round((1590+ 1590+1960+ 1670 +1610+ 1780+ 1420)/ 7 )
Onthego = round((1780 + 1880)/2) 
Onthego_lim = round((2280 +2800 +1920+ 1920+ 2050 +2370+ 2140+ 2610+ 2060 +2420+ 2620 )/ 11)
Metis_lim = round((1670+1670+ 1770+ 1860 )/4)

# Hermes
Kelly_25 = round((6800+ 6350+ 6650) /3)
Kelly_25_ost = 15300
Kelly_25_liz = 13800
Kelly_25_ali = 20300
Kelly_25_ali_h = 28900
Kelly_28 = round((6940+7280 )/2)
Kelly_28_ost = 15300
Kelly_28_ali = 28900
Kelly_32 = round((6800 +7050)/2)
Kelly_32_liz = 6600
Kelly_35 = 7150
Kelly_35_ost = 15400
Kelly_35_ali = 28900
Birkin_25 = round((6570 +6410)/2)
Birkin_25_ost = 13800
Birkin_25_ali = 20300
Birkin_25_ali_h = 28900
Birkin_30 = round((7510 + 8600 +7700)/3)
Birkin_30_ost = 13800
Birkin_30_ali = 28900
Birkin_35 = round((8300+7100)/2)
Birkin_35_ost = 15400
Birkin_35_ali = 28900
Birkin_40 = round((8770+ 9250)/2)
Birkin_40_ost = 15400
Birkin_40_ali = 28900


# Chanel 
Timeless_Classique = 4340
Timeless_Classique_h = 5800
Timeless_Classique_exot = 8040
Timeless_Classique_ali = 27600
Boy = round((2620+3200+2840+3480+2740+3890+2690+4170+3890)/9)
Boy_exotic = 5400
Boy_ali = 22000
Woc = round((2060 +1790 + 1120 +990)/4)
two_55 = round((5040 +4830+ 5550+5300)/4)
two_55_ali = 14200

# bags['mean_retail_price'] = np.nan

In [ ]:
# Assign values to the right models of each brand category

bags.loc[(bags['limited'] == 0) & (bags['m_model'] == 'Model: Speedy'), 'mean_retail_price'] = Speedy
bags.loc[(bags['m_model'] == 'Model: Speedy Bandoulière'), 'mean_retail_price'] = Speedy_lim
bags.loc[(bags['m_model'] == 'Model: Speedy') & (bags['m_material'] == 'Material: Exotic leathers'), 'mean_retail_price'] = Speedy_py
bags.loc[(bags['limited'] == 1) & (bags['m_model'] == 'Model: Speedy time trunk'), 'mean_retail_price'] = Speedy_t
bags.iloc[71, 15] = Speedy_ost
bags.iloc[3483, 15] = Neverfull_ali
bags.loc[(bags['limited'] == 1) & (bags['m_model'] == 'Model: Neverfull'), 'mean_retail_price'] = Neverfull_lim
bags.loc[(bags['m_model'] == 'Model: Neverfull'), 'mean_retail_price'] = Neverfull
bags.loc[(bags['m_material'] == 'Material: Leather') & (bags['m_model'] == 'Model: Alma'), 'mean_retail_price'] = Alma_l
bags.loc[(bags['limited'] == 1) & (bags['m_model'] == 'Model: Alma'), 'mean_retail_price'] = Alma_lim
bags.loc[(bags['m_material'] == 'Material: Patent leather') & (bags['m_model'] == 'Model: Alma BB'), 'mean_retail_price'] = Alma_bb_l
bags.loc[(bags['limited'] == 0) & (bags['m_model'] == 'Model: Capucines'), 'mean_retail_price'] = Capucines
bags.loc[(bags['limited'] == 1) & (bags['m_model'] == 'Model: Capucines'), 'mean_retail_price'] = Capucines_lim
bags.loc[(bags['m_material'] == 'Material: Exotic leathers') & (bags['m_model'] == 'Model: Capucines'), 'mean_retail_price'] = Capucines_ali
bags.loc[(bags['price'] < 500) & (bags['m_model'] == 'Model: Pochette'), 'mean_retail_price'] = Pochette
bags.loc[(bags['retail_price'].isnull()) & (bags['m_model'] == 'Model: Twist'), 'mean_retail_price'] = Twist
bags.loc[(bags['m_material'] == 'Material: Exotic leathers') & (bags['m_model'] == 'Model: Twist'), 'mean_retail_price'] = Twist_py
bags.loc[(bags['m_material'] == 'Material: Cloth') & (bags['m_model'] == 'Model: Pochette Accessoire'), 'mean_retail_price'] = Pochette
bags.loc[(bags['limited'] == 1) & (bags['m_model'] == 'Model: Pochette'), 'mean_retail_price'] = Pochette_lim
bags.loc[(bags['price'] > 500) & (bags['m_model'] == 'Model: Pochette'), 'mean_retail_price'] = Pochette_lim
bags.loc[(bags['limited'] == 1) & (bags['m_model'] == 'Model: Metis'), 'mean_retail_price'] = Metis_lim
bags.loc[(bags['m_model'] == 'Model: NéoNoé_bb'), 'mean_retail_price'] = NéoNoé_bb
bags.loc[(bags['limited'] == 1) & (bags['m_model'] == 'Model: NéoNoé'), 'mean_retail_price'] = NéoNoé_lim

bags.loc[(bags['m_model'] == 'Model: Kelly 25'), 'mean_retail_price'] = Kelly_25
bags.loc[bags['title'] == 'Kelly 25 ostrich handbag', 'mean_retail_price'] = Kelly_25_ost
bags.loc[bags['title'] == 'Kelly 25 alligator handbag', 'mean_retail_price'] = Kelly_25_ali
bags.loc[(bags['title'] == 'Kelly 25 lizard handbag'), 'mean_retail_price'] = Kelly_25_liz
bags.loc[(bags['title'] == 'Kelly 28 lizard handbag'), 'mean_retail_price'] = Kelly_28_liz
bags.loc[bags['m_model'] == 'Model: Kelly 35', 'mean_retail_price'] = Kelly_35
bags.loc[bags['title'] == 'Burgundy Lizard Handbag Kelly 20', 'mean_retail_price'] = Kelly_25
bags.loc[(bags['m_material'] == 'Material: Exotic leathers') & (bags['m_model'] == 'Kelly 35'), 'mean_retail_price'] = Birkin_30_ali
bags.loc[(bags['m_model'] == 'Model: Kelly 25'), 'mean_retail_price'] = Kelly_25
bags.loc[bags['title'] == 'Kelly 32 alligator handbag', 'mean_retail_price'] = Kelly_32_ali
bags.loc[bags['title'] == 'Kelly 35 alligator handbag', 'mean_retail_price'] = Kelly_35_ali
bags.loc[bags['title'] == 'Birkin 30 ostrich handbag', 'mean_retail_price'] = Birkin_30_ost
bags.loc[bags['title'] == 'Birkin 30 alligator handbag', 'mean_retail_price'] = Birkin_30_ali
bags.loc[bags['title'] == 'Birkin 35 ostrich handbag', 'mean_retail_price'] = Birkin_35_ost
bags.loc[bags['title'] == 'Birkin 35 alligator handbag', 'mean_retail_price'] = Birkin_35_ali
bags.loc[(bags['m_model'] == 'Model: Birkin 40'), 'mean_retail_price'] = Birkin_40
bags.loc[bags['title'] == 'Birkin 40 ostrich handbag', 'mean_retail_price'] = Birkin_40_ost
bags.loc[bags['title'] == 'Birkin 40 alligator handbag', 'mean_retail_price'] = Birkin_40_ali

bags.loc[(bags['m_model'] == 'Model: Boy'), 'retail_price'] = Boy
bags.loc[(bags['m_material'] == 'Material: Exotic leathers') & (bags['m_model'] == 'Model: Boy'), 'retail_price'] = Boy_exotic
bags.loc[(bags['title'] == 'Timeless/Classique alligator handbag'), 'retail_price'] = Timeless_Classique_ali
bags.loc[(bags['price'] < 3600) & (bags['m_model'] == 'Model: Timeless/Classique'), 'retail_price'] = Timeless_Classique
bags.loc[(bags['price'] > 3600) & (bags['m_model'] == 'Model: Timeless/Classique'), 'retail_price'] = Timeless_Classique_h
bags.loc[(bags['m_material'] == 'Material: Exotic leathers') & (bags['m_model'] == 'Model: Timeless/Classique'), 'retail_price'] = Timeless_Classique_exot
bags.loc[(bags['m_model'] == 'Model: 2.55'), 'retail_price'] = two_55
bags.loc[(bags['m_material'] == 'Material: Exotic leathers') & (bags['m_model'] == 'Model: 2.55'), 'retail_price'] = two_55_ali
bags.loc[(bags['m_model'] == 'Model: Woc') | (bags['m_model'] == 'Model: Wallet on Chain'), 'retail_price'] = Woc


## Preliminary Data Munging and Cleaning

In [295]:
# Read previously created file and review the first 5 rows of dataframe
bags = pd.read_csv('preview.csv')
bags.head()

,brand,title,price_range,price,discount_price,n_likes,limited,model,condition,m_model,material,color,m_material,sold_out,description,mean_retail_price
0,Louis Vuitton,Pochette Accessoire cloth clutch bag,"£1,080 £776 £776",NaN,NaN,53.0,1,NaN,Never worn,Model: Pochette Accessoire,NaN,Color: Brown,Material: Cloth,1.0,NaN,660.0
1,Louis Vuitton,Pochette Accessoire cloth clutch bag,£495 £495,NaN,NaN,42.0,1,NaN,Never worn,Model: Pochette Accessoire,NaN,Color: Brown,Material: Cloth,1.0,NaN,660.0
2,Louis Vuitton,Speedy cloth handbag,"£1,480 £1480",NaN,NaN,52.0,1,NaN,Never worn,Model: Speedy,NaN,Color: Pink,Material: Cloth,1.0,NaN,1790.0
3,Louis Vuitton,Speedy cloth handbag,"£1,880 £1880",NaN,NaN,59.0,1,NaN,Very good condition,Model: Speedy,NaN,Color: White,Material: Cloth,1.0,NaN,1790.0
4,Louis Vuitton,Speedy cloth handbag,"£1,850 £1,680 £1680",NaN,NaN,60.0,1,NaN,Very good condition,Model: Speedy,NaN,Color: Red,Material: Cloth,1.0,NaN,1790.0


In [296]:
# defining information about our data, checking types and non-null values
bags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12916 entries, 0 to 12915
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brand              12308 non-null  object 
 1   title              12916 non-null  object 
 2   price_range        12916 non-null  object 
 3   price              0 non-null      float64
 4   discount_price     0 non-null      float64
 5   n_likes            12916 non-null  float64
 6   limited            12916 non-null  int64  
 7   model              0 non-null      float64
 8   condition          12916 non-null  object 
 9   m_model            12292 non-null  object 
 10  material           0 non-null      float64
 11  color              12916 non-null  object 
 12  m_material         12916 non-null  object 
 13  sold_out           12916 non-null  float64
 14  description        5127 non-null   object 
 15  mean_retail_price  12774 non-null  float64
dtypes: float64(7), int64(1

In [297]:
# Assing null values in brand column to fix it later on
bags.loc[bags['brand'].isnull(), 'brand'] = bags['title']

In [298]:
# check if any nan values has left
bags[bags.brand.isnull()]

,brand,title,price_range,price,discount_price,n_likes,limited,model,condition,m_model,material,color,m_material,sold_out,description,mean_retail_price


In [299]:
# Check the valid inputs in column
bags.color.unique()

array(['Color: Brown', 'Color: Pink', 'Color: White', 'Color: Red',
       'Color: Camel', 'Color: Multicolour', 'Color: Orange',
       'Color: Blue', 'Color: Anthracite', 'Color: Purple', 'Color: Ecru',
       'Color: Green', 'Color: Black', 'Color: Burgundy', 'Color: Other',
       'Color: Beige', 'Color: Silver', 'Color: Navy', 'Color: Khaki',
       'Color: Gold', 'Color: Yellow', 'Color: Metallic', 'Color: Grey',
       'Location:France,\n          from\n          the seller Vestiaire Collective',
       'Color: Turquoise',
       'Location:United States,\n          from\n          the seller willow',
       'Color: brown/black',
       'Location:Italy,\n          from\n          the seller ludovica',
       'Color: camel'], dtype=object)

In [300]:
# Review invalid inpute
bags[(bags['color'] == 'Location:France,\n          from\n          the seller Vestiaire Collective') | (bags['color'] == 'Location:United States,\n          from\n          the seller willow') | (bags['color'] == 'Location:Italy,\n          from\n          the seller ludovica')]

,brand,title,price_range,price,discount_price,n_likes,limited,model,condition,m_model,material,color,m_material,sold_out,description,mean_retail_price
3881,Louis Vuitton,Neverfull cloth tote,£659 £523.16 £523.16,NaN,NaN,64.0,0,NaN,Good condition,Model: Neverfull,NaN,"Location:France,\n from\n th...",Material: Cloth,1.0,NaN,1070.0
5591,Louis Vuitton,Alma patent leather handbag - Grayish Blue,£914.12 £837.94 £837.94,NaN,NaN,29.0,0,NaN,Very good condition,Model: Alma,NaN,"Location:United States,\n from\n ...",Material: Patent leather,1.0,NaN,1670.0
10856,Hermès,Kelly 32 leather handbag,"£5,164.72 £5164.72",NaN,NaN,20.0,0,NaN,Good condition,Model: Kelly 32,NaN,"Location:Italy,\n from\n the...",Material: Leather,1.0,"Kelly 32 handbag in dark taupe Togo leather, e...",6925.0
12428,Hermès,Leather Handbag Birkin 40,"£6,350.14 £6350.14",NaN,NaN,46.0,0,NaN,Very good condition,Model: Birkin 40,NaN,"Location:France,\n from\n th...",Material: Leather,1.0,"Birkin 40 Retourné tote bag in Togo leather, ""...",9010.0


In [301]:
# Fix manually colors based on the description and info from website
bags.iloc[3881, 11] = 'Color: Brown'
bags.iloc[5591, 11] = 'Color: Blue'
bags.iloc[10856, 11] = 'Color: Brown'
bags.iloc[12428, 11] = 'Color: Black'

In [302]:
# Extract unique model name with lambda function and substitute nan value for further assigning step

bags.m_model = bags.m_model.apply(lambda x: str(x).split(':')[1].strip() if str(x) != 'nan' else np.nan)

In [303]:
# Check results and count the subgroups
bags.m_model.value_counts()

Speedy                 2022
Neverfull              1579
Boy                    1358
Alma                   1339
Timeless/Classique      798
Birkin 35               780
Wallet on Chain         767
2.55                    645
Kelly 32                546
Birkin 30               425
Alma BB                 370
Kelly 28                338
Kelly 35                330
Birkin 40               186
Pochette Accessoire     166
Kelly 25                160
Birkin 25               152
Speedy Bandoulière      144
Keepall                  37
Capucines                33
Twist                    32
Onthego                  28
Metis                    27
NéoNoé                   23
Speedy time trunk         3
NéoNoé BB                 2
Alma Graffiti             2
Name: m_model, dtype: int64

In [304]:
# Check invalid inputs and manually replace them with valid values
bags.title.unique() 

array(['Pochette Accessoire cloth clutch bag', 'Speedy cloth handbag',
       'Speedy leather bowling bag', 'Neverfull cloth tote',
       'Keepall cloth 48h bag', 'Neverfull cloth handbag',
       'Capucines leather handbag',
       'Speedy Bandoulière cloth crossbody bag', 'NéoNoé cloth handbag',
       'Onthego leather tote', 'Alma patent leather handbag',
       'Pochette Accessoire leather clutch bag', 'Twist leather handbag',
       'Speedy Bandoulière shearling handbag',
       'Speedy Bandoulière cloth handbag', 'Alma cloth handbag',
       'Speedy glitter bowling bag', 'Capucines alligator handbag',
       'Neverfull leather tote', 'Metis cloth crossbody bag',
       'Pochette Accessoire leather bag', 'Keepall cloth travel bag',
       'Neverfull leather clutch bag', 'Neverfull shearling tote',
       'NéoNoé cloth bag', 'NéoNoé leather handbag',
       'Speedy leather handbag', 'Twist leather crossbody bag',
       'Neverfull leather handbag', 'Speedy cloth bag',
       'Onth

In [305]:
bags.title = bags.title.apply(lambda x: x.replace('\\"Alma\\" leather handbag', 'Alma leather bag').replace('Tote', 'Neverfull').replace('in...', '').replace("There are marks on the flap please see photos", '')
                              .replace('\tCloth', ' Cloth').replace('Leather satchel', 'Alma Leather satchel').replace(' Patent Leather satchel', 'Alma Patent Leather satchel').replace('Velvet handbag', 'Boy Velvet handbag')
                             .replace('Leather mini bag', 'Timeless/Classique Leather mini bag').replace('Leather bag', 'Kelly Leather bag').replace(' 48h', '').replace('24h ', '')
                             .replace('. "".', '').replace('Timeless/Classique Timeless/Classique Timeless/Classique Timeless/Classique Timeless/Classique Timeless/Classique Timeless/Classique Leather mini bag', 'Timeless/Classique Leather mini bag')
                             .replace('  bag', 'bag').replace('2.55 LEATHER Kelly 32 leather handbag', '2.55 leather handbag').replace('Gris Etain PHW ', '')
                             .replace('Timeless/Classique Timeless/Classique Timeless/Classique Leather mini bag', 'Timeless/Classique Leather mini bag').replace('Beige ', '').strip())


In [306]:
# reassign a few rows that left with uninformative description 

bags.iloc[6869, 1] = 'Boy leather handbag'
bags.iloc[8781, 1] = '2.55 leather handbag'
bags.loc[(bags.title == 'BAG'), 'title'] = 'Kelly 32 leather handbag'

In [307]:
# Create lambda function to extract first word from title column and assign missing values in m_model

f = lambda x: x['title'].split(' ')[0].strip()
bags.loc[bags.m_model.isnull(), 'm_model'] = bags.apply(f, axis=1)

In [308]:
bags.m_model.value_counts()

Speedy                 2098
Neverfull              1654
Boy                    1421
Alma                   1400
Timeless/Classique      862
Birkin 35               780
Wallet on Chain         767
2.55                    706
Kelly 32                546
Birkin 30               425
Alma BB                 370
Kelly 28                338
Kelly 35                330
Birkin 40               186
Pochette Accessoire     166
Kelly 25                160
Birkin 25               152
Speedy Bandoulière      144
Kelly                    68
Wallet                   62
Birkin                   61
Keepall                  37
Capucines                35
Twist                    32
Metis                    31
Onthego                  31
NéoNoé                   26
Pochette                 17
Speedy time trunk         3
NéoNoé BB                 2
Alma Graffiti             2
Burgundy                  1
Cloth                     1
Silver                    1
Leather                   1
Name: m_model, dtype

In [309]:
# bags.loc[bags['m_model'] == 'Burgundy']

# Substitute invalid values with the appropriate ones manulally
bags.loc[bags['m_model'] == 'Leather', 'm_model'] = 'Neverfull'
bags.loc[bags['m_model'] == 'Silver', 'm_model'] = 'Speedy'
bags.loc[bags['m_model'] == 'Cloth', 'm_model'] = 'Speedy'
bags.loc[bags['m_model'] == 'Burgundy', 'm_model'] = 'Kelly'
bags.loc[bags['m_model'] == 'Wallet', 'm_model'] = 'Woc'

In [310]:
# bags.loc[bags.title == 'Leather crossbody bag']
# bags.loc[bags.title == 'Leather tote']
# bags.loc[bags.title == 'Handbag']

In [311]:
# Fix some groups of title column with relevant values (checked previously the subsets of these rows)

funct1 = lambda x: str(x['m_model'] + ' Leather crossbody bag')
bags.loc[bags.title == 'Leather crossbody bag', 'title'] = bags.apply(funct1, axis=1)

funct2 = lambda x: str(x['m_model'] + ' Leather tote')
bags.loc[bags.title == 'Leather tote', 'title'] = bags.apply(funct2, axis=1)

funct3 = lambda x: str(x['m_model'] + ' handbag')
bags.loc[bags.title == 'Handbag', 'title'] = bags.apply(funct3, axis=1)


In [312]:
# Assign model column to specific model name by extracting certain value with lambds function

l = lambda x: str(x['m_model']).replace('Wallet', 'woc').replace('Timeless/Classique', 'classic_flap').lower().split(' ')[0].strip()
bags['model'] = bags.apply(l, axis=1)

In [313]:
# Check results and count values of each subcategory
bags.model.value_counts()

speedy          2247
alma            1772
neverfull       1655
birkin          1604
kelly           1443
boy             1421
classic_flap     862
woc              829
2.55             706
pochette         183
keepall           37
capucines         35
twist             32
metis             31
onthego           31
néonoé            28
Name: model, dtype: int64

In [314]:
#  Make 3 lists out of unique names in brand column in chronological order assign each value to relevant brand name
# bags.brand.unique()

list_l = ['Silver Cloth Handbag Speedy',
       'Capucines leather handbag',
       'Pochette Accessoire cloth clutch bag',
       'Neverfull patent leather handbag', 'Onthego cloth tote',
       'Speedy leather bowling bag', 'Metis cloth handbag',
       'Speedy cloth handbag', 'Speedy leather handbag',
       'NéoNoé cloth handbag', 'Neverfull leather tote',
       'NéoNoé leather handbag', 'Neverfull leather clutch bag',
       'Neverfull cloth tote', 'Speedy handbag',
       'Pochette Accessoire clutch bag', 'Metis cloth crossbody bag',
       'Pochette Accessoire leather clutch bag', 'Onthego cloth handbag',
       'Neverfull cloth handbag', 'Neverfull patent leather bag',
       'Alma leather handbag', 'Speedy cloth bowling bag',
       'Metis leather crossbody bag', 'Speedy glitter handbag',
       'NéoNoé cloth bag', 'Neverfull cloth mini bag',
       'Speedy cloth tote', 'Speedy leather bag', 'Speedy cloth bag',
       'Cloth handbag', 'Neverfull leather handbag',
       'Neverfull cloth bag', 'Neverfull tote', 'Neverfull leather bag',
       'Neverfull patent leather tote', 'Beige Leather Handbag Neverfull',
       'Alma patent leather handbag', 'Alma BB leather bag',
       'Alma cloth handbag', 'Alma BB leather handbag',
       'Alma BB cloth crossbody bag', 'Alma BB patent leather handbag',
       'Alma BB cloth handbag', 'Alma BB patent leather bag',
       'Alma leather bag', 'Alma BB patent leather crossbody bag',
       'Alma cloth bag', 'Alma patent leather bowling bag',
       'Alma BB leather satchel', 'Alma leather crossbody bag']

list_c = ['Boy leather handbag', 'Boy leather bag',
       'Boy leather crossbody bag', 'Boy stingray mini bag',
       'Boy handbag', 'Boy tweed handbag', 'Boy lizard handbag',
       'Boy tweed crossbody bag', 'Boy patent leather crossbody bag',
       'Boy velvet handbag', 'Boy glitter crossbody bag',
       'Boy python bag', 'Boy leather mini bag',
       'Timeless/Classique leather crossbody bag',
       'Timeless/Classique leather handbag',
       'Timeless/Classique leather bag',
       'Timeless/Classique glitter handbag',
       'Timeless/Classique patent leather handbag',
       'Timeless/Classique linen handbag',
       'Timeless/Classique leather mini bag',
       'Timeless/Classique velvet bag',
       'Timeless/Classique cloth crossbody bag', '2.55 leather handbag',
       '2.55 handbag', '2.55 crossbody bag',
       '2.55 patent leather handbag', '2.55 leather crossbody bag',
       '2.55 leather bag', '2.55 patent leather mini bag',
       '2.55 patent leather bag', '2.55 alligator handbag',
       '2.55 cloth handbag', '2.55 tweed handbag',
       'Wallet on Chain leather handbag',
       'Wallet on Chain leather crossbody bag',
       'Wallet on Chain leather bag',
       'Wallet on Chain tweed crossbody bag',
       'Wallet on Chain python crossbody bag',
       'Wallet on Chain patent leather handbag',
       'Wallet on Chain velvet crossbody bag',
       'Wallet on Chain patent leather crossbody bag']

liss = ['Kelly 28 leather handbag', 'Burgundy Lizard Handbag Kelly 20',
       'Kelly 25 leather handbag', 'Kelly 32 leather handbag',
       'Kelly leather handbag', 'Kelly 25 leather bag',
       'Kelly 25 leather crossbody bag', 'Kelly 28 leather bag',
       'Kelly 35 leather handbag', 'Leather bag',
       'Birkin 35 leather handbag', 'Kelly 32 leather bag',
       'Kelly 32 cloth handbag', 'Kelly 25 leather tote',
       'Birkin 30 leather handbag', 'Birkin 25 leather handbag',
       'Birkin 35 cloth handbag', 'Birkin 30 leather tote',
       'Birkin 35 leather bag', 'Birkin 30 ostrich handbag',
       'Birkin 25 leather bag', 'Birkin 25 leather tote',
       'Birkin 40 leather handbag', 'Birkin 40 leather bag',
       'Birkin 35 leather tote']

bags.brand = bags.brand.apply(lambda x:'Louis Vuitton' if x in list_l else x)

In [315]:
bags.brand = bags.brand.apply(lambda x:'Chanel' if x in list_c else x)

In [316]:
bags.brand = bags.brand.apply(lambda x:'Hermès' if x in liss else x)

In [317]:
# Check results after implementing lambda functions on this brand column and count values in all subcategories
bags.brand.value_counts()

Louis Vuitton    6051
Chanel           3818
Hermès           3047
Name: brand, dtype: int64

In [318]:
# Clean color column from prefix 'Color:' and from duplicates color, keep only unique color names
bags.color = bags.color.apply(lambda x: x.replace('camel', 'Camel').replace('brown/black', 'Black').split(':')[1].strip())

bags.color.value_counts()

Brown          3616
Black          2801
Blue            834
Red             695
Multicolour     573
Beige           559
Pink            529
Other           506
White           422
Grey            337
Green           285
Burgundy        272
Orange          238
Camel           221
Purple          212
Navy            172
Gold            164
Yellow          132
Silver          123
Ecru             64
Turquoise        50
Metallic         44
Khaki            39
Anthracite       28
Name: color, dtype: int64

In [319]:
# Clean material column from prefix 'Material:', keep only unique material names / drop m_material column
bags.material = bags.m_material.apply(lambda x: x.split(':')[1].strip())
bags.material.value_counts()

Leather                7683
Cloth                  3515
Patent leather          762
Exotic leathers         380
Synthetic               130
Velvet                   77
Cotton                   72
Tweed                    68
Denim - Jeans            62
Suede                    41
Fur                      31
Plastic                  25
Glitter                  18
Wool                     15
Silk                     11
Linen                     9
Pony-style calfskin       5
Faux fur                  4
Other                     4
Polyester                 2
Wicker                    1
Metal                     1
Name: material, dtype: int64

In [320]:
bags.drop(['m_material'], axis=1, inplace=True)

In [321]:
# Fill price column with the main price from price_range column , convert it to float and remove unrelevant characters
bags.price = bags.price_range.apply(lambda x: float(x.replace('£', '').replace(',', '').split(' ')[0]))
bags.price

0         1080.00
1          495.00
2         1480.00
3         1880.00
4         1850.00
           ...   
12911    13783.90
12912    14296.80
12913    12330.99
12914     5986.79
12915    14261.35
Name: price, Length: 12916, dtype: float64

In [322]:
# Fill discount_price column with discounted price where relevant from price_range column 
# Convert it to float and remove unneeded characters, check the results after implemented lambda function

bags.discount_price = bags.price_range.apply(lambda x: float(x.replace('£', '').replace(',', '').split(' ')[1]))
bags.discount_price

0          776.00
1          495.00
2         1480.00
3         1880.00
4         1680.00
           ...   
12911    13783.90
12912    14296.80
12913    10445.60
12914     4512.43
12915    14261.35
Name: discount_price, Length: 12916, dtype: float64

In [323]:
# Clean very low values, many of them has price typos 
bags.drop(bags.index[(bags.discount_price < 200)], inplace=True)

# reset index and dropping new column with old index
bags.reset_index(inplace=True, drop=True)

In [324]:
# Check if material column Exotic leather category wrongly assigned 

bags.loc[bags['title'].str.contains("Crocodile|crocodile|Alligator|alligator") & (bags.material == 'Leather')]

,brand,title,price_range,price,discount_price,n_likes,limited,model,condition,m_model,material,color,sold_out,description,mean_retail_price
8696,Chanel,2.55 alligator handbag,"£14,499 £13,395 £13395",14499.00,13395.00,42.0,0,2.55,Very good condition,2.55,Leather,Black,1.0,NaN,14200.0
10477,Hermès,Kelly 35 crocodile handbag,"£9,829.05 £8,256.4 £8256.4",9829.05,8256.40,90.0,0,kelly,Very good condition,Kelly,Leather,Black,0.0,NaN,28900.0
11083,Hermès,Kelly 28 crocodile handbag,"£11,314.25 £8,849.71 £8849.71",11314.25,8849.71,79.0,0,kelly,Very good condition,Kelly 28,Leather,Navy,1.0,"Year 2017, limited edition. ""Kelly 28"" model h...",28900.0


In [325]:
# Assign it to the right material categoty (because of big difference in price it's better to fix) 

bags.loc[bags['title'].str.contains("Crocodile|crocodile|Alligator|alligator"), 'material'] = 'Exotic leathers'

In [326]:
# Manually final cleaning and assign some updated values for some obsevations in Lv and Hermes brands 
bags.iloc[970, 1] = 'Speedy cloth handbag Royal Wedding'
bags.iloc[970, 5] = 58
bags.iloc[970, 6] = 1
bags.iloc[970, 14] = 1790

bags.iloc[943, 6] = 1
bags.iloc[943, 14] = 1790
bags.iloc[4867, 6] = 1

bags.loc[(bags.discount_price > 1600) & bags['model'].str.contains("speedy"), 'limited'] = 1
bags.loc[(bags.discount_price > 1750) & bags['model'].str.contains("neverfull"), 'limited'] = 1
bags.loc[(bags.discount_price > 2700) & bags['model'].str.contains("speedy") & (bags['mean_retail_price'] == 825), 'mean_retail_price'] = 1640
bags.loc[(bags.discount_price > 2100) & bags['model'].str.contains("neverfull") & (bags['mean_retail_price'] == 1070), 'mean_retail_price'] = 1510

bags.iloc[11375, 10] = 'Exotic leathers'
bags.iloc[11375, 14] = 28900
bags.iloc[12333, 10] = 'Exotic leathers'
bags.iloc[12333, 14] = 20300
bags.iloc[10631, 9] = 'Kelly 25'
bags.iloc[10634, 9] = 'Kelly 32'
bags.iloc[10668, 9] = 'Kelly 25'

In [327]:
# bags.iloc[12196:12254]

In [328]:
# Create lambda function to extract first word from title column and assign it more descriptive in m_model column
f = lambda x: x['title'][:8].strip()
bags.loc[bags.m_model.str.endswith("Kelly"), 'm_model'] = bags.apply(f, axis=1)

f2 = lambda x: x['title'][:9].strip()
bags.loc[bags.m_model.str.endswith("Birkin"), 'm_model'] = bags.apply(f2, axis=1)

In [329]:
# drop price_range column (no need in it after extracting data in 'price' and 'discount_price' columns)

bags.drop(['price_range'], axis=1, inplace=True)

In [330]:
# rename one column name to make it more descriptive 
bags = bags.rename(columns={'discount_price': 'final_resale_price'})

In [331]:
# Cleaned data from invalid inputs and null values except description column (no need to have it for all bags because 
# of repeated information or not that informative for further steps)
bags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12774 entries, 0 to 12773
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   brand               12774 non-null  object 
 1   title               12774 non-null  object 
 2   price               12774 non-null  float64
 3   final_resale_price  12774 non-null  float64
 4   n_likes             12774 non-null  float64
 5   limited             12774 non-null  int64  
 6   model               12774 non-null  object 
 7   condition           12774 non-null  object 
 8   m_model             12774 non-null  object 
 9   material            12774 non-null  object 
 10  color               12774 non-null  object 
 11  sold_out            12774 non-null  float64
 12  description         5122 non-null   object 
 13  mean_retail_price   12774 non-null  float64
dtypes: float64(5), int64(1), object(8)
memory usage: 1.4+ MB


In [332]:
# Save cleaned data to csv file, keep same index
bags.to_csv('cleaned_data_handbags.csv', index=False)

## Data Dictionary :

* brand  - Brand Name

* title  - Title of the Handbag model

* price  - Secondary market price

* final_resale_price - Discounted price of secondary market if relevant (could be the same as initial price if there is no discount on the handbag 

* n_likes - Number of likes per handbag
* limited - Limited edition or not (yes is 1/ no is 0) 
* model   - More general name of the Model of handbag
* condition - Condition of the handbag
* m_model - Specific Name of the Model of handbag 
* material - Type of the material the handbag made of 
* color   -  Color of the handbag
* sold_out - Sold or Not sold (binary)
* description - Description of the handbag
* mean_retail_price - Mean Retail Price from original store or official articles
    